In [85]:
import pandas as pd
import numpy as np 
import requests

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

r = requests.get(uri)

with open('dados/data.csv', 'wb') as f:
    f.write(r.content)

In [86]:
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [87]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

SEED = 204035
np.random.seed(SEED)

x = dados.drop('vendido', axis = 1)
y = dados.vendido

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify = y)
print(f'Modelo treinado com {len(x_train)} dados e testado com {len(x_test)}')


modelo = LinearSVC()
modelo.fit(x_train, y_train)

acuracia = modelo.score(x_test, y_test)
print(f'A acuracia do modelo LinearSVC foi de {acuracia * 100 : 0.2f}%')

Modelo treinado com 7500 dados e testado com 2500
A acuracia do modelo LinearSVC foi de  58.00%


In [88]:
from sklearn.dummy import DummyClassifier

SEED = 204035
np.random.seed(SEED)

x = dados.drop('vendido', axis = 1)
y = dados.vendido

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify = y)
print(f'Modelo treinado com {len(x_train)} dados e testado com {len(x_test)}')


modelo = DummyClassifier()
modelo.fit(x_train, y_train)

acuracia = modelo.score(x_test, y_test)
print(f'A acuracia do modelo Dummy foi de {acuracia * 100 : 0.2f}%')


Modelo treinado com 7500 dados e testado com 2500
A acuracia do modelo Dummy foi de  52.32%


In [89]:
from sklearn.tree import DecisionTreeClassifier

SEED = 204035
np.random.seed(SEED)

x = dados.drop('vendido', axis = 1)
y = dados.vendido

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify = y)
print(f'Modelo treinado com {len(x_train)} dados e testado com {len(x_test)}')

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(x_train, y_train)

acuracia = modelo.score(x_test, y_test)
print(f'A acuracia do modelo DecisionTree foi de {acuracia * 100 : 0.2f}%')


Modelo treinado com 7500 dados e testado com 2500
A acuracia do modelo DecisionTree foi de  76.76%


In [90]:
from sklearn.model_selection import cross_validate

SEED = 204035
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x, y, cv = 5)
media = resultados['test_score'].mean()
desvio_padrao = resultados['test_score'].std()
print(f'Acuracia com cross validation, 5 = [{(media - 2* desvio_padrao) * 100 : 0.2f}, {(media + 2* desvio_padrao) * 100 : 0.2f}]')

Acuracia com cross validation, 5 = [ 75.21,  76.35]


# Aleatoriedade no cross validate

In [91]:
def imprime_resultados(results):
    media = resultados['test_score'].mean()
    desvio_padrao = resultados['test_score'].std()
    print(f'Acuracia média: {media * 100:0.2f}')
    print(f'Acuracia intervalo: [{(media - 2* desvio_padrao) * 100 :0.2f}, {(media + 2* desvio_padrao) * 100 :0.2f}]')

In [92]:
from sklearn.model_selection import KFold

SEED = 204035
np.random.seed(SEED)

cv = KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x, y, cv = cv)
imprime_resultados(resultados)

Acuracia média: 75.78
Acuracia intervalo: [74.37, 77.19]


In [93]:
SEED = 204035
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x, y, cv = cv)
imprime_resultados(resultados)

Acuracia média: 75.78
Acuracia intervalo: [73.49, 78.07]


# Simualar situação horrível de azar
Pode ser 'azar' como também pode ser uma situação de desbalanceamento dos dados

In [94]:
dados_azar = dados.sort_values('vendido')

x_azar = dados_azar.drop('vendido', axis = 1)
y_azar = dados_azar.vendido
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [95]:
SEED = 204035
np.random.seed(SEED)

cv = KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x_azar, y_azar, cv = cv)
imprime_resultados(resultados)

Acuracia média: 57.84
Acuracia intervalo: [34.29, 81.39]


In [96]:
SEED = 204035
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x_azar, y_azar, cv = cv)
imprime_resultados(resultados)

Acuracia média: 75.78
Acuracia intervalo: [73.37, 78.19]


In [97]:
from sklearn.model_selection import StratifiedKFold

SEED = 204035
np.random.seed(SEED)

cv = StratifiedKFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x_azar, y_azar, cv = cv)
imprime_resultados(resultados)

Acuracia média: 75.76
Acuracia intervalo: [73.20, 78.32]


# Validação por agrupamento
Criação de um modelo aleatorio para cada carro baseando-se em um intervalo de -2 a 2 a partir da idade do veiculo


In [98]:

np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2,3, size = 10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,19
1,40557.96,1,20,12622.05362,23
2,89627.50,0,12,11440.79806,13
3,95276.14,0,3,43167.32682,7
4,117384.68,1,4,12770.11290,7


In [99]:
from sklearn.model_selection import GroupKFold

SEED = 204035
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x_azar, y_azar, cv = cv, groups= dados.modelo)
imprime_resultados(resultados)

Acuracia média: 75.79
Acuracia intervalo: [74.17, 77.41]


# Cross validation com StandardScaler

In [103]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

modelo = SVC()
modelo.fit(x_train_scaled, y_train)

acuracia = modelo.score(x_test_scaled, y_test)
print(f'A acuracia do modelo SVC foi de {acuracia * 100 : 0.2f}%')

A acuracia do modelo SVC foi de  77.24%


In [107]:
SEED = 5798
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = SVC()
resultados = cross_validate(modelo, x_azar, y_azar, cv = cv, groups= dados.modelo)
imprime_resultados(resultados)

Acuracia média: 77.32
Acuracia intervalo: [75.64, 79.00]


In [113]:
from sklearn.pipeline import Pipeline


SEED = 5798
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])


cv = GroupKFold(n_splits=10)
resultados = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups= dados.modelo)
imprime_resultados(resultados)

Acuracia média: 76.76
Acuracia intervalo: [75.55, 77.98]
